### **## Python Performance Tuning**

**Timing python code**


1.   Command line
2.   Python Interface



In [ ]:
#timeit module - Measure execution time of small code snippets
#reference - https://docs.python.org/3/library/timeit.html
# default 1 million
# best of - average of least 5 timing recorded


'''
for cmd 
python -m timeit 3+3
'''
%timeit 3+3

The slowest run took 88.38 times longer than the fastest. This could mean that an intermediate result is being cached.
100000000 loops, best of 5: 12.8 ns per loop


In [ ]:
import timeit

def multiply_func(x):
    return x * x

%timeit multiply_func(3)

#%timeit -n 10 -r 2 multiply_func(3)

#-n how many times to execute ‘statement’
#-r best(average) x of the loop (default 5)

The slowest run took 10.64 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 113 ns per loop


In [ ]:
#globals() - this will cause the code to be executed within your current global namespace.
#this is for convenience so that no specific imports are needed

import timeit

totalTime = timeit.timeit('"-".join(str(n) for n in range(100))', number=10000)
#x = "-".join(str(n) for n in range(1000))
#print(x)

print(totalTime/10000)

2.5577715700001135e-05


**### Data Structure Choice: List Dictionary**

### **List**

**- Searching for an element in list takes O(N)**

In [ ]:
#simple search example

letters = 'ASD' #FGHJKLQWERTYUIOPZXCVBNM'

letters_list = [x + y + z 
                for x in letters
                  for y in letters
                    for z in letters]

print(letters_list[5:])

['ASD', 'ADA', 'ADS', 'ADD', 'SAA', 'SAS', 'SAD', 'SSA', 'SSS', 'SSD', 'SDA', 'SDS', 'SDD', 'DAA', 'DAS', 'DAD', 'DSA', 'DSS', 'DSD', 'DDA', 'DDS', 'DDD']


In [ ]:
# search xxx in the list
# note that ABC is not the first element but AAA
# here since -r is not mentioned it will take default best of 5

%timeit -n 100 'ABC' in letters_list 

%timeit -n 100 'PQR' in letters_list

## **Plot search time vs size of the list**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print(np.random.randint(0, 3, 2))

In [ ]:
import matplotlib.pyplot as plt

mycode = '''
def check_membership(elem):
    return elem in numbers_list
check_membership(i)
'''
times = []
for i in range(0,100000,10):
    numbers_list = np.random.randint(0, i, i)
    total_time = timeit.timeit(mycode, number = 5, globals=globals())
    
    times.append(total_time / 5)

plt.plot(times)
plt.xlabel('Size of the list')
plt.ylabel('Lookup time')
plt.show()

### **Dictionary**
- Dictionaries are implemented as a Hash table, which hash the key
- _Dicts_ and _sets_ are fast when looking up elements. 
- Insert, search and delete operations are O(1)

In [ ]:
letters_dict = {x: x for x in letters_list}
# Time how long it takes to find ‘abc’ and 'pqr'in letters_dict.

print(letters_dict)

print('in dict')
%timeit -n 100 'ABC' in letters_dict
%timeit -n 100 'PQR' in letters_dict

In [ ]:
mycode = '''
def check_membership(elem):
    return elem in numbers_dict
check_membership(i)
'''
times = []
for i in range(0,10000,10):
    numbers_list = np.random.randint(0,i,i)
    numbers_dict = {k:k for k in numbers_list}
    total_time = timeit.timeit(mycode, number = 5, globals=globals())
    
    times.append(total_time/5)


plt.plot(times)
plt.xlabel('Size of the dictionary')
plt.ylabel('Lookup time')

**reference https://towardsdatascience.com/which-python-data-structure-should-you-use-fa1edd82946c#:~:text=OOP%20requires%20the%20use%20of,tuples%2C%20dictionaries%2C%20and%20sets.**

**List** 
- mutable
- ordered 
- used to save data

**Tuple**
- immutable
- ordered

**Set**
- unordered
- unique data

**Dictionary**
- implemented as a hashtable
- search, delete, update operations

### **Function Choice: String Concatenation**

- Python strings are immutable.
- str1 + str2 creates a new string.
- This copying can lead to significant slowdown (use''.join instead)
- https://towardsdatascience.com/do-not-use-to-join-strings-in-python-f89908307273

In [ ]:
def method1():
    out_str = ''
    global loop_count
    for num in range(loop_count):
        out_str += 'num'
    return out_str

'''
def method2():
    str_list = []
    global loop_count
    for num in range(loop_count):
        str_list.append('num')
    return ''.join(str_list)
'''
#range() 0 1 2 3 4 5..... passed_param

def method3():
    global loop_count
    return ''.join(['num' for i in range(loop_count)])

loop_count = 100000

%timeit -n 10 method1()
#%timeit -n 10 method2()
%timeit -n 10 method3()

### **Optimizing loops**

- Avoid for loops, use map or numpy operations
- Numpy is faster due to vectorized implementations

Multiply two 100x100 matrices

In [ ]:
N = 100
matrix1 = np.random.random((N,N))
matrix2 = np.random.random((N,N))

### **How fast is  For loop ?**

- multiplication of 2 matrix

In [ ]:
%%timeit -n 1

def multiply(x,y):

    m1,n1 = x.shape
    m2,n2 = y.shape
    
    assert(n1 == m2)
    z = np.zeros((m1,n2))

    for i in range(m1): 
        for j in range(n2): 
            for k in range(m2): 
                z[i][j] += x[i][k] * y[k][j]
                
    return z

multiply(matrix1, matrix2)


1 loop, best of 5: 1.45 s per loop


### **How fast is numpy?**

- we check this by multiplying the same arrays.
- why is numpy faster
  - NumPy uses a highly-optimized, carefully-tuned BLAS method for matrix multiplication. The specific function in this case is GEMM (for generic matrix multiplication).https://stackoverflow.com/questions/10442365/why-is-matrix-multiplication-faster-with-numpy-than-with-ctypes-in-python
  - BLAS algorithm - https://www.quora.com/What-algorithm-does-BLAS-use-for-matrix-multiplication-Of-all-the-considerations-e-g-cache-popular-instruction-sets-Big-O-etc-which-one-turned-out-to-be-the-primary-bottleneck

In [ ]:
%%timeit -n 1

def mod_multiply(x,y):
    """
    Multiply two arrays using numpy.
    """
    return np.matmul(x,y)

mod_multiply(matrix1, matrix2)

### **Decorators**

In Python, functions are the first class objects, which means that:

- Functions are objects; they can be referenced to, passed to a variable and returned from other functions as well.

- Functions are taken as the argument into another function and then called inside the wrapper function.

- easily add functionality to an existing function by adding to wrapper that functionality functions

- https://www.youtube.com/watch?v=FsAPt_9Bf3U


In [ ]:
# defining a decorator, pass in "func", return "wrapper"

def my_decorator(func):
    
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_hi():
    print("Hi!")

say_hi()

print()

say_hi = my_decorator(say_hi)
say_hi()

In [ ]:
@my_decorator
def say_whee():
    print("Whee!")
say_whee()

In [ ]:
## Showing args and kwargs for decorators
#kwargs - key word arguments
def decorator_with_args(func):
    
    def wrapper(*args, **kwargs):
        print("Something is happening before the function is called.")
        func(*args, **kwargs)
        print("Something is happening after the function is called.")
    return wrapper


@decorator_with_args
def say_hi_withargs(*args, **kwargs):
    for arg in args:
      print(arg)
    if 'course' in kwargs:
        print('Course: ', kwargs['course'])
        
say_hi_withargs('Welcome to ', 'Lab 2',course='Python')